In [2]:
import tkinter as tk
from math import sqrt

In [10]:
class SudokuSolver3D:
    def __init__(self, window):
        self.window = window
        self.window.title("3D Sudoku Solver")
        self.window.geometry("1000x600")

        self.set_up_window_1()

### window 1 ###        
    def set_up_window_1(self):
        
        def set_up_top(self):
            self.frm_top = tk.Frame(pady = 5)
            lbl_title = tk.Label(master=self.frm_top, text='3D Sudoku Solver', font = ('TkDefaultFont', 15))
            lbl_title.pack()
            self.frm_top.grid(row = 0, column= 0, columnspan= 2)
        set_up_top(self)

        def set_up_board(self):
            # creating sudoku board
            self.frm_board = tk.Frame()

            self.x_offset = 0
            self.y_offset = 0
            self.cnv_board = tk.Canvas(master = self.frm_board, bd =2, width = '600', height = '500')

            self.side_length = 120

            self.faces = []
            self.face_shapes = []
            self.lines = [[] for x in range(4)] 

            # starting face
            self.faces.append(Face('top', (300,150), self.side_length))

            self.face_shapes.append(self.cnv_board.create_polygon(self.faces[0].vertices, fill = 'white', outline = 'black', activefill='honeydew1'))
            self.cnv_board.tag_bind(self.face_shapes[-1], '<Button-1>', self.choose_face_to_add)

            # add_face(self.faces[0], 'left', 'below') #1
            # add_face(self.faces[0], 'right', 'below') #2
            # add_face(self.faces[0], 'top', 'above_left') #3
            # add_face(self.faces[0], 'left', 'above') #4
            # add_face(self.faces[2], 'left', 'right') #5
            # add_face(self.faces[4], 'right', 'right') #6
            # add_face(self.faces[6], 'left', 'right') #7
            # add_face(self.faces[7], 'top', 'below') #8
            # add_face(self.faces[8], 'top', 'below_right') #9

            self.cnv_board.pack()
            self.frm_board.grid(row = 1, column= 0, padx = 15, pady = 5)

            self.btn_start_board = tk.Button(master = self.frm_board, text = "Start with board", font=('TkDefaultFont', 10))
            self.btn_start_board.bind("<Button-1>", self.start_board_click)
            self.btn_start_board.pack()

        set_up_board(self)

        def set_up_controls(self):
            self.frm_controls = tk.Frame(height = 500, width = 300, highlightbackground='black', highlightthickness=1)
            self.frm_controls.pack_propagate(False)

            self.cnv_buttons = tk.Canvas(master= self.frm_controls, height = 80, width = 300)

            self.buttons = [Face('top', (50,40), 50), Face('left', (150,40), 50), Face('right', (250,40), 50)]

            self.button_shapes = []
            for button in self.buttons:
                self.button_shapes.append(self.cnv_buttons.create_polygon(button.vertices, fill = 'white', outline = 'black', activeoutline='darkseagreen3'))
                self.cnv_buttons.tag_bind(self.button_shapes[-1], '<Button-1>', self.choose_where_add)

            self.frm_controls.grid(row = 1, column = 1, padx = 15)
        set_up_controls(self)

    
    def choose_face_to_add(self, event):
        for face_shape in self.face_shapes:
            self.cnv_board.itemconfigure(face_shape, fill = 'white')
        
        for line in self.lines:
            self.cnv_board.delete(line)
        
        for button_shape in self.button_shapes:
            self.cnv_buttons.itemconfigure(button_shape, outline = 'black')


        item = self.cnv_board.find_withtag("current")[0]
        self.cnv_board.itemconfigure(item, fill = 'honeydew1')

        self.existing_face = self.faces[self.face_shapes.index(item)]
        
        self.cnv_buttons.pack()
   
    def choose_where_add(self, event):
        for line in self.lines:
            self.cnv_board.delete(line)

        for button_shape in self.button_shapes:
            self.cnv_buttons.itemconfigure(button_shape, outline = 'black')

        item = self.cnv_buttons.find_withtag("current")[0]
        self.cnv_buttons.itemconfigure(item, outline = 'darkseagreen3')

        self.picked_button = self.buttons[self.button_shapes.index(item)]
        self.new_type = self.picked_button.type
        
        if self.existing_face.type == self.new_type:
            indices = [0,1,2,3]
        elif (set([self.existing_face.type, self.new_type]) == set(['left', 'right'])) or self.new_type == 'right': 
            indices = [0,2]
        else:
            indices = [1,3]

        for index in indices:
            if self.existing_face.neighbours[index] == None:
                self.lines[index] = (self.cnv_board.create_line(self.existing_face.edges[index], width = 2, capstyle = 'round', activefill = 'darkseagreen3'))
                self.cnv_board.tag_bind(self.lines[index], '<Button-1>', self.add_new_face)

    def add_new_face(self, event):
        item = self.cnv_board.find_withtag("current")[0]
        self.cnv_board.itemconfigure(item, fill = 'darkseagreen3')

        index = self.lines.index(item)

        if self.existing_face.type in set(['left', 'right']):
            if index == 0:
                self.new_location =  'left'
            elif index == 1:
                self.new_location =  'below'
            elif index == 2:
                self.new_location =  'right'
            elif index == 3:
                self.new_location =  'above'
                
        elif self.existing_face.type == 'top':
            if index == 0:
                self.new_location =  'left above'
            elif index == 1:
                self.new_location =  'left below'
            elif index == 2:
                self.new_location =  'right below'
            elif index == 3:
                self.new_location =  'right above'

                
        self.new_face(self.existing_face, self.new_type, self.new_location)

        ## scroll canvas
        self.cnv_board.configure(scrollregion = self.cnv_board.bbox("all"))
        if self.cnv_board.find_overlapping(self.x_offset,self.y_offset, 600 + self.x_offset, self.y_offset):
            self.y_offset += 120
        if self.cnv_board.find_overlapping(self.x_offset, 500 + self.y_offset, 600 + self.x_offset, 500 + self.y_offset):
            self.y_offset -= 120
        if self.cnv_board.find_overlapping(self.x_offset,self.y_offset, self.x_offset, 500 + self.y_offset):
            self.x_offset += 50
        if self.cnv_board.find_overlapping(600 + self.x_offset,self.y_offset, 600 + self.x_offset, 500 + self.y_offset):
            self.x_offset -= 50

        ## reset everything
        self.cnv_board.itemconfigure(self.face_shapes[self.faces.index(self.existing_face)], fill = 'white')
        self.cnv_buttons.itemconfigure(self.button_shapes[self.buttons.index(self.picked_button)], outline = 'black')
        self.cnv_buttons.pack_forget()
        for line in self.lines:
            self.cnv_board.delete(line)
        
        ## update neighbours 
        for face in self.faces[:-1]:
            shared_edges = list(set(self.faces[-1].edges) & set(face.edges))
            if len(shared_edges) == 1:
                self.faces[-1].neighbours[self.faces[-1].edges.index(shared_edges[0])] = self.faces.index(face)
                face.neighbours[face.edges.index(shared_edges[0])] = len(self.faces) - 1
        
    def new_face(self, existing_face, new_type, new_location):

        side_length = self.side_length        
        existing_type = existing_face.type
        existing_centre = existing_face.centre
        
        if new_type == 'top':
            if existing_type == 'top':
                dx = -(side_length * sqrt(3)/ 2) if 'left' in new_location else (side_length * sqrt(3)/ 2)
                dy = -(side_length/2) if 'above' in new_location else (side_length/2)
            
            elif existing_type == 'left':
                dx = (side_length* sqrt(3)/ 4) if 'above' in new_location else -(side_length* sqrt(3)/ 4)
                dy = -(side_length * 3/4) if 'above' in new_location else (side_length * 3/4)

            elif existing_type == 'right':
                dx = -(side_length* sqrt(3)/ 4) if 'above' in new_location else (side_length* sqrt(3)/ 4)
                dy = -(side_length * 3/4) if 'above' in new_location else (side_length * 3/4)
    
            self.faces.append(Face('top', (existing_centre[0]+dx, existing_centre[1]+dy), side_length))

        elif new_type == 'left':
            if existing_type == 'top':   
                dx = (side_length* sqrt(3)/ 4) if 'above' in new_location else -(side_length* sqrt(3)/ 4)
                dy = -(side_length * 3/4) if 'above' in new_location else (side_length * 3/4)

            elif existing_type == 'left':
                if new_location in set(['above', 'below']):
                    dx = 0
                    dy = -side_length if new_location == 'above' else side_length
                elif new_location in set(['left', 'right']):
                    dx = -(side_length * sqrt(3)/ 2) if new_location == 'left' else (side_length * sqrt(3)/ 2)
                    dy = -(side_length/2) if new_location == 'left' else (side_length/2)

            elif existing_type == 'right':
                dx = -(side_length * sqrt(3)/ 2) if new_location == 'left' else (side_length * sqrt(3)/ 2)
                dy = 0
            
            self.faces.append(Face('left', (existing_centre[0]+dx, existing_centre[1]+dy), side_length))

        elif new_type == 'right':
            if existing_type == 'top':
                dx = -(side_length* sqrt(3)/ 4) if 'above' in new_location else (side_length* sqrt(3)/ 4)
                dy = -(side_length * 3/4) if 'above' in new_location else (side_length * 3/4)
            
            elif existing_type == 'left':
                dx = -(side_length * sqrt(3)/ 2) if new_location == 'left' else (side_length * sqrt(3)/ 2)
                dy = 0

            elif existing_type == 'right':
                if new_location in set(['above', 'below']):
                    dx = 0
                    dy = -side_length if new_location == 'above' else side_length
                elif new_location in set(['left', 'right']):
                    dx = -(side_length * sqrt(3)/ 2) if new_location == 'left' else (side_length * sqrt(3)/ 2)
                    dy = (side_length/2) if new_location == 'left' else -(side_length/2)
            
            self.faces.append(Face('right', (existing_centre[0]+dx, existing_centre[1]+dy), side_length))

        self.face_shapes.append(self.cnv_board.create_polygon(self.faces[-1].vertices, fill = 'white', outline = 'black', activefill = 'honeydew1'))
        self.cnv_board.tag_bind(self.face_shapes[-1], '<Button-1>', self.choose_face_to_add)

    def start_board_click(self,event):

        if self.check_3d_board_valid:
            pass
        else:
            print('error')

    def check_3d_board_valid(self):
        for face_index in range(len(self.faces)):
            face = self.faces[face_index]
            for i in range(4):
                if face.neighbours[i] is not None:
                    neighbouring_face_index =  self.faces[face_index].neighbours[i]
                    neighbouring_face = self.faces[neighbouring_face_index]
                    if (face.neighbours[i-2] is not None) != (neighbouring_face.neighbours[neighbouring_face.neighbours.index(face_index)-2] is not None):
                        return False
        return True

class Face:
    def __init__(self, type, centre, side_length):
        self.type = type
        self.centre = centre
        self.side_length = side_length
        if self.type == 'top':
            self.vertices = [centre[0], centre[1]-(side_length/2), 
                             centre[0]-(side_length * sqrt(3)/ 2), centre[1], 
                             centre[0], centre[1]+(side_length/2),
                             centre[0]+(side_length * sqrt(3)/ 2), centre[1]]
            
        elif self.type == 'left': 
            self.vertices = [centre[0]-(side_length * sqrt(3)/ 4), centre[1]-(side_length* 3/4), 
                             centre[0]-(side_length * sqrt(3)/ 4), centre[1]+(side_length /4), 
                             centre[0]+(side_length * sqrt(3)/ 4), centre[1]+(side_length*3/4),
                             centre[0]+(side_length * sqrt(3)/ 4), centre[1]-(side_length /4)]

        elif self.type == 'right':
            self.vertices = [centre[0]-(side_length * sqrt(3)/ 4), centre[1]-(side_length/4), 
                             centre[0]-(side_length * sqrt(3)/ 4), centre[1]+(side_length * 3/4), 
                             centre[0]+(side_length * sqrt(3)/ 4), centre[1]+(side_length/4),
                             centre[0]+(side_length * sqrt(3)/ 4), centre[1]-(side_length * 3/4)]
            
        self.edges = []
        for start, end in zip([0,2,4,6], [2,4,6,0]):
            vertices = sorted([(self.vertices[start], self.vertices[start+1]), (self.vertices[end], self.vertices[end+1])], key = lambda element: (element[0], element[1]))
            self.edges.append(tuple(vertices))
        
        self.neighbours =  [None,None,None,None]


## white, honeydew1, honeydew2


In [11]:
if __name__ == "__main__":
    window = tk.Tk()
    app = SudokuSolver3D(window)
    window.mainloop() 

In [15]:
print((None is None) == (None is None))


True


In [3]:
tk.Entry?

Init signature: tk.Entry(master=None, cnf={}, **kw)
Docstring:      Entry widget which allows displaying simple text.
Init docstring:
Construct an entry widget with the parent MASTER.

Valid resource names: background, bd, bg, borderwidth, cursor,
exportselection, fg, font, foreground, highlightbackground,
highlightcolor, highlightthickness, insertbackground,
insertborderwidth, insertofftime, insertontime, insertwidth,
invalidcommand, invcmd, justify, relief, selectbackground,
selectborderwidth, selectforeground, show, state, takefocus,
textvariable, validate, validatecommand, vcmd, width,
xscrollcommand.
File:           c:\program files\windowsapps\pythonsoftwarefoundation.python.3.11_3.11.2544.0_x64__qbz5n2kfra8p0\lib\tkinter\__init__.py
Type:           type
Subclasses:     

In [ ]:
        for row in self.grid_rows:
            for pos in list(row):
                self.cnv_board.itemconfigure(self.shp_square[pos[0]][pos[1]], fill = 'honeydew1')
            self.window.update()
            self.window.after(300,self.window.update())
            for pos in list(row):
                self.cnv_board.itemconfigure(self.shp_square[pos[0]][pos[1]], fill = 'white')
